# CPD

In [49]:
# Bibliothèques necessaires
import cv2 
import face_recognition
import pandas as pd
import numpy as np
from matplotlib import pyplot

In [50]:
# Paramètres
vid = cv2.VideoCapture(0) 
face_locations = []
face_encodings = []

# Enregistrement de personnes identifiable
paul_image = face_recognition.load_image_file("Documents/photo_paul.jpg")
amoka_image = face_recognition.load_image_file("Documents/photo_amoka.jpg")
romain_image = face_recognition.load_image_file("Documents/photo_romain.jpg")

paul_face_encoding = face_recognition.face_encodings(paul_image)[0]
amoka_face_encoding = face_recognition.face_encodings(amoka_image)[0]
romain_face_encoding = face_recognition.face_encodings(romain_image)[0]


known_face_encodings = [paul_face_encoding, amoka_face_encoding, romain_face_encoding]
known_face_names = ["Paul", "Amoka", "Romain"]

In [51]:
# Fonction qui défini le nom d'une personne si elle est reconnu
def person_name(face_encodings, face_names):
    for face_encoding in face_encodings: 
        name = "Unknown"
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        face_names.append(name)
    return face_names

In [52]:
#     # Enregistre la frame et l'affiche
def screen_video():
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    pyplot.imshow(gray_frame, cmap='gray')
    pyplot.axis('off')
    pyplot.show()

In [53]:
# Afficher les FPS
import time

def show_fps(frame, prev_frame_time):
    specific_frame = []
    font = cv2.FONT_HERSHEY_SIMPLEX
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 
    fps = int(fps)
    fps = str(fps)
    cv2.putText(frame, fps, (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return prev_frame_time

In [54]:
def face_detection():
    # Fabrique une plus petite frame pour booster les performances
    face_names = []
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Detecte et ajoute a une liste les visages sur la "petite" frame
    face_locations = face_recognition.face_locations(rgb_small_frame)    
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    # Charger la liste
    face_names = person_name(face_encodings, face_names)
    
    # pour chaque visage detecté
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Carré autour du visage
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Ajoute le nom
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

In [55]:
prev_frame_time = 0

# Boucle vidéo
while(True): 
    ret, frame = vid.read() 
    face_detection()
#     prev_frame_time = show_fps(frame, prev_frame_time)
    cv2.imshow('frame', frame)
#     screen_video()
    if cv2.waitKey(1) & 0xFF == ord('q'):        
        break

vid.release() 
cv2.destroyAllWindows()